# HW 11 AmirHossein Naghdi 400102169


# 15 Points on the notebook running correctly.

# 15 Points on having sufficient explanations and overall readability of the notebook

In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, ResNet50
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_data = np.concatenate((x_train, x_test))
y_data = np.concatenate((y_train, y_test))
x_data = x_data.astype("float32") / 255.0


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


# 10 Points: Creating a convolutional network with Keras (with at least two layers of convolution layer)

In [2]:
def run_cross_validation(build_model, preprocess=None, epochs=5, batch_size=64):
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    acc_list = []

    for train_idx, val_idx in kf.split(x_data):
        x_train, x_val = x_data[train_idx], x_data[val_idx]
        y_train_, y_val_ = y_data[train_idx], y_data[val_idx]

        y_train = tf.keras.utils.to_categorical(y_train_, 10)
        y_val = tf.keras.utils.to_categorical(y_val_, 10)

        if preprocess:
            x_train = preprocess(x_train)
            x_val = preprocess(x_val)

        model = build_model()
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
        _, acc = model.evaluate(x_val, y_val, verbose=0)
        acc_list.append(acc)

    print(f"Average Accuracy: {np.mean(acc_list):.4f}")
    return np.mean(acc_list)


In [3]:

def build_cnn(kernel_size=3):
    model = models.Sequential([
        layers.Conv2D(32, (kernel_size, kernel_size), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (kernel_size, kernel_size), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

for k in [3, 5, 7]:
    print(f"\nTesting kernel size {k}")
    run_cross_validation(lambda: build_cnn(kernel_size=k))



Testing kernel size 3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Average Accuracy: 0.6445

Testing kernel size 5
Average Accuracy: 0.6418

Testing kernel size 7
Average Accuracy: 0.5652


# 20 Points: Tuning the above network for:
* 5 Points: Tuning the kernel size (i.e. the size of the receptive field) for convolutional layers
* 5 Points: Tuning the stride for convolutional layers
* 5 Points: Tuning the pooling size (i.e. the size of the receptive field) for pooling layers
* 5 Points: Tuning the stride for pooling layers

In [4]:
def build_cnn_advanced(conv_stride=1, pool_size=2, pool_stride=2):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), strides=conv_stride, activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D(pool_size=(pool_size, pool_size), strides=pool_stride),
        layers.Conv2D(64, (3, 3), strides=conv_stride, activation='relu'),
        layers.MaxPooling2D(pool_size=(pool_size, pool_size), strides=pool_stride),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

configs = [
    (1, 2, 2),
    (2, 2, 2),
    (1, 3, 2)
]

for conv_stride, pool_size, pool_stride in configs:
    print(f"\nStride={conv_stride}, Pool Size={pool_size}, Pool Stride={pool_stride}")
    run_cross_validation(lambda: build_cnn_advanced(conv_stride, pool_size, pool_stride))



Stride=1, Pool Size=2, Pool Stride=2
Average Accuracy: 0.6628

Stride=2, Pool Size=2, Pool Stride=2
Average Accuracy: 0.4612

Stride=1, Pool Size=3, Pool Stride=2
Average Accuracy: 0.6489


# 10 Points: Perform data augmentation and train your model above using the ImageGenerator class

In [5]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

def build_augmented_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

kf = KFold(n_splits=3, shuffle=True)
for train_idx, val_idx in kf.split(x_data):
    x_train, x_val = x_data[train_idx], x_data[val_idx]
    y_train = tf.keras.utils.to_categorical(y_data[train_idx], 10)
    y_val = tf.keras.utils.to_categorical(y_data[val_idx], 10)

    model = build_augmented_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=5, verbose=0)
    _, acc = model.evaluate(x_val, y_val, verbose=0)
    print("Augmented Fold Accuracy:", acc)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Augmented Fold Accuracy: 0.6376000046730042
Augmented Fold Accuracy: 0.6402000188827515
Augmented Fold Accuracy: 0.628849983215332


# 20 Points: Perform transfer learning using two of the available models in Keras applications (e.g. VGG19, ResNet, EfficientNet, etc.)

In [6]:
def build_transfer_model(base_model, preprocess):
    base = base_model(include_top=False, weights='imagenet', input_shape=(32, 32, 3))
    base.trainable = False

    model = models.Sequential([
        layers.Lambda(lambda x: preprocess(x)),
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

print("\nTransfer Learning with VGG19")
run_cross_validation(lambda: build_transfer_model(VGG19, vgg_preprocess))

print("\nTransfer Learning with ResNet50")
run_cross_validation(lambda: build_transfer_model(ResNet50, resnet_preprocess))



Transfer Learning with VGG19
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Average Accuracy: 0.2297

Transfer Learning with ResNet50
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Average Accuracy: 0.3291


np.float64(0.32910000284512836)

# 10 Points: Express you opnion about the effects of the window size (i.e. receptive field) in convolution layers on the performance of neural network. In other words, what happens if we increase or decrease the size of the receptive field? and Why?

## What is the effect of kernel (receptive field) size on performance?

Increasing the receptive field (kernel size) can help capture more global features,
but may lead to overfitting on small datasets or loss of local detail. Smaller kernels
are more effective in early layers, capturing fine textures. Larger kernels can be
useful in deeper layers where abstract features are learned.

Balance is key: too small → underfitting, too large → overfitting or inefficient.
